In [1]:
import pandas as pd
#import sklearn 
#import tensorflow as tf

In [ ]:
def load_csv(path):     
    csv_path = path
    if csv_path != "":
        df = pd.read_csv(csv_path)
        print(df)
        return df
    
    else:
        return None

In [ ]:
DUMP_FILE_PATH = "S:\\SOAR\\Uploads\\exported.csv"
df = load_csv(DUMP_FILE_PATH)